In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
milestones = pd.read_csv('data/nycMilestones.csv')
# milestones.shape

# # https://data.cityofnewyork.us/resource/s7yh-frbm

milestones["orig_start_date"] = pd.to_datetime(milestones['orig_start_date'])
milestones["orig_end_date"] = pd.to_datetime(milestones['orig_end_date'])
milestones["task_start_date"] = pd.to_datetime(milestones['task_start_date'])
milestones["task_end_date"] = pd.to_datetime(milestones['task_end_date'])

# (milestones['task_start_date'] - milestones['orig_start_date']) / np.timedelta64(1, 'M')\
small = pd.DataFrame()
small[["project_id","managing_agency","boro","orig_start","actual_start","step","task"]] = milestones[["project_id","managing_agcy","boro","orig_start_date","task_start_date","seq_number","task_description"]]

# milestones[milestones['task_start_date'] < milestones['orig_start_date']].head()
small["starting_delta"]= ((milestones.task_start_date - milestones.orig_start_date)/np.timedelta64(1, 'M'))
small["ending_delta"]= ((milestones.task_end_date - milestones.orig_end_date)/np.timedelta64(1, 'M'))

small.to_csv('data/small.csv')


In [14]:
numProjects = small["project_id"].nunique()

# Q1 - Are Projects *Actually* Delayed?

In [32]:
# using the small dataframe, group by project and select the max ending_delta and starting_delta
late = small.groupby('project_id').agg({'ending_delta': 'max', 'starting_delta': 'max'})

# How many projects are completed late?
lateFinishProjects = late[late['ending_delta'] > 0].reset_index().shape[0]
lateFinishProjects /  numProjects

# How many projects are started late?
lateStartProjects = late[late['starting_delta'] > 0].reset_index().shape[0]
lateStartProjects /  numProjects

# How many projects are started late and finished late?
lateStartFinishProjects = late[(late['starting_delta'] > 0) & (late['ending_delta'] > 0)].reset_index().shape[0]
lateStartFinishProjects /  numProjects

# How many projects are finished early?
earlyFinishProjects = late[late['ending_delta'] < 0].reset_index().shape[0]
earlyFinishProjects /  numProjects

# How many projects are started early?
earlyStartProjects = late[late['starting_delta'] < 0].reset_index().shape[0]
earlyStartProjects /  numProjects

# How many projects are started early and finished early?
earlyStartFinishProjects = late[(late['starting_delta'] < 0) & (late['ending_delta'] < 0)].reset_index().shape[0]
earlyStartFinishProjects /  numProjects

# How many projects are started early and finished late?
earlyStartLateFinishProjects = late[(late['starting_delta'] < 0) & (late['ending_delta'] > 0)].reset_index().shape[0]
earlyStartLateFinishProjects /  numProjects

# How many projects are started on time?
onTimeStartProjects = late[late['starting_delta'] == 0].reset_index().shape[0]
onTimeStartProjects /  numProjects

# How many projects are finished on time?
onTimeFinishProjects = late[late['ending_delta'] == 0].reset_index().shape[0]
onTimeFinishProjects /  numProjects

# How many projects are started on time and finished on time?
onTimeStartFinishProjects = late[(late['starting_delta'] == 0) & (late['ending_delta'] == 0)].reset_index().shape[0]
onTimeStartFinishProjects /  numProjects

# How many projects are started on time and finished late?
onTimeStartLateFinishProjects = late[(late['starting_delta'] == 0) & (late['ending_delta'] > 0)].reset_index().shape[0]
onTimeStartLateFinishProjects /  numProjects

# How many projects are started late and finished on time?
onTimeStartLateFinishProjects = late[(late['starting_delta'] > 0) & (late['ending_delta'] == 0)].reset_index().shape[0]
onTimeStartLateFinishProjects /  numProjects

# How many projects are started late and finished early?
earlyStartLateFinishProjects = late[(late['starting_delta'] > 0) & (late['ending_delta'] < 0)].reset_index().shape[0]
earlyStartLateFinishProjects /  numProjects

0.0017837235228539577

## Q1 Answer: Yes
--> 62.15% of projects were completed late  
--> 36.58% of projects were started late  
--> 36.19% of projects were both started and completed late  

--> 1.5% of projects are finished early  
--> 2.5% of projects are finished early  
--> 1.3% of projects are started and finished early  
--> 1.2% of projects are started early and finished late  

--> 60.87% of projects are started on time  
--> 36.3% of projects are completed on time  
--> 24.8% of projects are started on time but completed late  
--> 0.2% of projects are started late and finished on time  
--> 0.18% of projects are started late and finished early  

Plots
- side by side boxplots of time deltas by start and end on time, early, and late
- heat map of where time deltas are
- 

#Q2 - How much do these projects cost?
- Capital Project Detail Dollars [link](https://data.cityofnewyork.us/City-Government/Capital-Project-Detail-Data-Dollars/wa2y-rh4b)

filter the dollars data by projects that are completed 
- add buckets for months? 
- 


In [33]:
small[small["project_id"]=="L103RENO"]

,project_id,managing_agency,boro,orig_start,actual_start,step,task,starting_delta,ending_delta
3053,L103RENO,NEW YORK RESEARCH LIBRARY,CITYWIDE,2022-10-10,2022-06-11,1,DESIGN,-3.975441,9.035093
3054,L103RENO,NEW YORK RESEARCH LIBRARY,CITYWIDE,2022-03-11,2022-12-11,2,BID AWARD AND REGISTER CONTRCT,9.035093,-2.989794
3055,L103RENO,NEW YORK RESEARCH LIBRARY,CITYWIDE,2022-06-11,2022-03-12,3,CONSTRUCTION,-2.989794,-2.989794
3056,L103RENO,NEW YORK RESEARCH LIBRARY,CITYWIDE,2022-06-12,2022-03-13,4,"PUNCHLIST COMPLETE, JOB CLOSED",-2.989794,-5.585330


## Avg Project Task Delay by Managing Agency

In [ ]:
# # Vertical bar
# small.groupby('managing_agency').mean().plot(kind='bar');
# plt.title('Average Project Delay (Months) \nby Managing Agency');
# plt.ylabel('Months');
# plt.xlabel('Managing Agency');
# plt.xticks(rotation=45,ha='right');
# plt.legend(['Starting Delay','Ending Delay'],loc='upper left', bbox_to_anchor=(-0.5, 1));

# Horizontal bar
small.groupby('managing_agency').mean().plot(kind='barh');
plt.title('Average Task Delay (Months) \nby Managing Agency');
plt.ylabel('Managing Agency');
plt.xlabel('Months');
plt.legend(['Starting Delay','Ending Delay'],loc='upper right');
plt.savefig('plots/avg_delay_by_agency.png');

## Top 5 Projects (Late and Early)

In [ ]:
small[#( small['step'] == 1 ) &
        (small['task'] == 'CONSTRUCTION START') &
        ( small['starting_delay_m'] > 0 )
      ].sort_values(by='starting_delay_m',ascending=False).head(10)